In [21]:
# -*- coding: utf-8 -*-
"""
Created on Tue Aug 21 08:47:10 2018

@author: mrestrepo
@company: Yuxi Global (www.yuxiglobal.com)
"""
from collections import OrderedDict
from itertools import product
import random

import os
from importlib import reload

import pandas as pd
#import xgboost as xgb
#%%
import pandas as pd
import numpy as np

from memoize import Memoizer
from inst_func_eval import InstFunEvaluator

from sklearn.metrics import roc_auc_score
#from sklearn.ensemble import RandomForestClassifier
#%%

#%%

# Data preprocessed by:
# data-analytics/tree_classif_explain/mapi_presentation/HC_Default_Risk.ipynb

In [25]:
#%%
def main() :
    #%%
    #data0 = { "x_train" : pd.read_pickle( DATA_DIR + "/x_train.pkl" ),
    #         "y_train" : pd.read_pickle( DATA_DIR + "/y_train.pkl" ),
    #         "x_test"  : pd.read_pickle( DATA_DIR + "/x_test.pkl" ),
    #         "y_test"  : pd.read_pickle( DATA_DIR + "/y_test.pkl" ) }

    #%%
    param_grid_dic = OrderedDict([
            ("learning_rate", [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ),
            ("max_depth"    , [  3 , 4 , 5, 6,  8, 10, 12, 15 ] ),
            ("min_child_weight", [ 1, 3, 5, 7 ] ),
            ("gamma", [0.0, 0.1, 0.2, 0.3, 0.4 ]),
            ("colsample_bytree", [  0.3, 0.4, 0.5, 0.7 ] ),
            ])

    #%%
    train_fraction = 0.05
    test_fraction = 0.16
    #data = subsample( data0, train_fraction, test_fraction )
    data=None
    #%%
    seed=1359
    log_level=0
    #%%
    memoization_path = DATA_DIR + "/" + "xgboost_memo%g" % train_fraction
    print( "memoization_path= " + memoization_path)
    if not os.path.exists( memoization_path ) :
        os.mkdir( memoization_path )
    #%%
    fun = Memoizer( lambda param_dic : train_xgb( data, param_dic ),
                    memoization_path )
    #%%
    grid_search( param_grid_dic, fun )
    #%%


def run_trials( n_trials, param_grid_dic, fun, method, method_name ) :

    for i in range(n_trials) :
        print( "trial = %d" % i )
        fun_eval = method( param_grid_dic, fun, seed=i )

def grid_search( param_grid_dic, fun, seed, log_level=0 ) :
    #%%
    import inst_func_eval as ife

    param_combos_shuffled = make_param_combos( param_grid_dic, seed=seed)

    best_auc = 0
    #best_combo = None

    fun_eval = ife.InstFunEvaluator( fun, param_grid_dic )

    for i, param_dic  in enumerate( param_combos_shuffled ) :
        auc = fun_eval.eval_fun( param_dic )
        if log_level > 0 :
            print( tuple( param_dic.items() )  )
            print( auc,  " best_auc: ", best_auc  )
        if auc > best_auc :
            best_auc = auc
            #best_combo = param_dic

    #%%
    return fun_eval


def coordinate_descent( param_grid_dic, fun, seed=1359 ) :
    #%%
    import coordescent as cd
    reload(cd)

    #param_grid = param_grid_dic.values()

    fun_min = lambda param_dic : -fun(param_dic)

    random.seed( seed  )

    best_val, best_idxs, fun_eval = cd.coordinate_descent( fun_min, param_grid_dic, x_idxs=None)
    #%%
    return fun_eval
    #%%
def genetic( param_grid_dic, fun, seed=1336 ) :
    #%%
    from importlib import reload
    import genetic as G
    reload( G )

    genes_grid = param_grid_dic

    best_val, best_idxs, fun_eval  = G.genetic_algorithm( fun,  genes_grid,
                                       init_pop = None, pop_size = 10, n_gen=30,
                                       mutation_prob=0.1,
                                       normalize = G.normalizer( 2.0, 0.01),
                                       seed=seed )

    # Params first batch
    #best_val, best_idxs, fun_eval  = G.genetic_algorithm( fun,  genes_grid,
    #                                   init_pop = None, pop_size = 30, n_gen=10,
    #                                   mutation_prob=0.1,
    #                                   normalize = G.normalizer( 2.0, 0.01),
    #                                   seed=seed )

    #print( best_val, fun_eval.eval_cnt() )
    #%%
    return fun_eval
    #%% 0.7407
    #_ = G.genetic_algorithm( fun, gene_names, genes_grid,
    #                                 init_pop = None, pop_size = 30, n_gen=10,
    #                                 mutation_prob=0.2,
    #                                 #normalize = g.normalizer( 1.0, 0.3),
    #                                 seed=1337 )
    #%%


def test( data, memoization_path ) :
    #%%
    param_dic = OrderedDict([('learning_rate', 0.2),
             ('max_depth', 5),
             ('min_child_weight', 1),
             ('gamma', 0.4),
             ('colsample_bytree', 0.7)])


    train_xgb( data, param_dic, memoization_path=None, model_type='xgb')

    #%%

def subsample( data0, train_fraction, test_fraction, seed=1337 ) :
    data = data0.copy()
    n_train = len(data0["x_train"])
    assert n_train== len(data0["y_train"])
    #%%
    np.random.seed( seed )
    r = np.random.rand(n_train) < train_fraction
    data["x_train"] = data0["x_train"].loc[r]
    data["y_train"] = data0["y_train"].loc[r]

    #%%
    np.random.seed( seed )
    n_test= len( data0["x_test"])
    assert n_test == len(data0["y_test"])

    r = np.random.rand(n_test) < test_fraction

    data["x_test"] = data0["x_test"].loc[r]
    data["y_test"] = data0["y_test"].loc[r]
    #%%
    return data


def make_param_combos( param_grid_dic, seed=1337 ) :
    #param_lens = [ len(v) for v  in param_grid_dic.values() ]
    param_names = list( param_grid_dic.keys() )

    param_idx_ranges = [ range(len(v)) for v in param_grid_dic.values() ]
    all_param_idx_combos = product( *param_idx_ranges  )
    all_param_combos = [  OrderedDict( ( name, param_grid_dic[name][idx])
                                for name, idx in  zip(param_names,idx_combo ) )
                          for idx_combo in all_param_idx_combos   ]
    #%%
    np.random.seed( seed )
    param_combos_shuffled = all_param_combos.copy()
    np.random.shuffle( param_combos_shuffled )
    #%%
    return param_combos_shuffled

#%%

def train_xgb( data, param_dic, model_type='xgb' ) :
    # fit model no training data

    if model_type == 'xgb' :
        model = xgb.XGBClassifier( learning_rate=param_dic["learning_rate"],
                                   max_depth=param_dic["max_depth"],
                                   min_child_weight=param_dic["min_child_weight"],
                                   colsample_bytree=param_dic["colsample_bytree"],
                                   gamma=param_dic["gamma"])
    else :
        model = RandomForestClassifier( n_estimators=100, min_samples_split=50, min_samples_leaf=10, max_depth=12)
    print( type(model) )
    model.fit(data["x_train"], data["y_train"] )

    y_pred = model.predict_proba( data["x_test"] )[:,1]
    auc = roc_auc_score( data["y_test"], y_pred )
    #
    return auc
#%%


In [29]:

#def run_trials( n_trials, param_grid_dic, fun, method, method_name ) :
n_trials=10000
method=genetic

dfs = []
for i in range(n_trials) :
        if i % 100 == 0 :
            print( "trial = %d" % i )
        fun_eval = method( param_grid_dic, fun, seed=i )
        df = pd.DataFrame( fun_eval.eval_log() ) 
        del df["pars"]
        df["method"] = method.__name__
        df["trial"] = i
        dfs.append( df )

trial = 0
trial = 100
trial = 200
trial = 300
trial = 400
trial = 500
trial = 600
trial = 700
trial = 800
trial = 900
trial = 1000
trial = 1100
trial = 1200
trial = 1300
trial = 1400
trial = 1500
trial = 1600
trial = 1700
trial = 1800
trial = 1900
trial = 2000
trial = 2100
trial = 2200
trial = 2300
trial = 2400
trial = 2500
trial = 2600
trial = 2700
trial = 2800
trial = 2900
trial = 3000
trial = 3100
trial = 3200
trial = 3300
trial = 3400
trial = 3500
trial = 3600
trial = 3700
trial = 3800
trial = 3900
trial = 4000
trial = 4100
trial = 4200
trial = 4300
trial = 4400
trial = 4500
trial = 4600
trial = 4700
trial = 4800
trial = 4900
trial = 5000
trial = 5100
trial = 5200
trial = 5300
trial = 5400
trial = 5500
trial = 5600
trial = 5700
trial = 5800
trial = 5900
trial = 6000
trial = 6100
trial = 6200
trial = 6300
trial = 6400
trial = 6500
trial = 6600
trial = 6700
trial = 6800
trial = 6900
trial = 7000
trial = 7100
trial = 7200
trial = 7300
trial = 7400
trial = 7500
trial = 7600
trial = 770

In [31]:
pd.concat( dfs ).to_hdf( "df_gen_10.hdf", "a/") 

C:\Users\Mateo\Miniconda3\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['extra', 'method']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [18]:
pd.DataFrame( fun_eval.eval_log() ) 

,eval_cnt,extra,fun_val,pars
0,0,None,0.708480,"{'learning_rate': 0.2, 'max_depth': 10, 'min_c..."
1,1,None,0.714631,"{'learning_rate': 0.3, 'max_depth': 3, 'min_ch..."
2,2,None,0.717157,"{'learning_rate': 0.25, 'max_depth': 4, 'min_c..."
3,3,None,0.691164,"{'learning_rate': 0.3, 'max_depth': 12, 'min_c..."
4,4,None,0.731409,"{'learning_rate': 0.1, 'max_depth': 5, 'min_ch..."
5,5,None,0.735515,"{'learning_rate': 0.1, 'max_depth': 4, 'min_ch..."
6,6,None,0.732538,"{'learning_rate': 0.1, 'max_depth': 6, 'min_ch..."
7,7,None,0.716736,"{'learning_rate': 0.1, 'max_depth': 12, 'min_c..."
8,8,None,0.738379,"{'learning_rate': 0.1, 'max_depth': 3, 'min_ch..."
9,9,None,0.717532,"{'learning_rate': 0.2, 'max_depth': 5, 'min_ch..."


In [8]:
fun = Memoizer( lambda param_dic : train_xgb( data, param_dic ),
                memoization_path )


Memoizer: Loaded cache from: ./xgboost_memo0.05/memos.pkl  with 3840 records 


In [4]:
DATA_DIR = r"."
param_grid_dic = OrderedDict([
        ("learning_rate", [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ),
        ("max_depth"    , [  3 , 4 , 5, 6,  8, 10, 12, 15 ] ),
        ("min_child_weight", [ 1, 3, 5, 7 ] ),
        ("gamma", [0.0, 0.1, 0.2, 0.3, 0.4 ]),
        ("colsample_bytree", [  0.3, 0.4, 0.5, 0.7 ] ),
        ])

#%%
train_fraction = 0.05
test_fraction  = 0.16
#data = subsample( data0, train_fraction, test_fraction )
data=None
#%%
seed=1359
log_level=0
#%%
memoization_path = DATA_DIR + "/" + "xgboost_memo%g" % train_fraction
print( "memoization_path= " + memoization_path)
if not os.path.exists( memoization_path ) :
    os.mkdir( memoization_path )
#%%
fun = Memoizer( lambda param_dic : train_xgb( data, param_dic ),
                memoization_path )
#%%
grid_search( param_grid_dic, fun )

memoization_path= ./xgboost_memo0.05


NameError: name 'Memoizer' is not defined